In [1]:
# ✅ Install Hugging Face Transformers, Datasets, and Evaluation Libraries
!pip install -q transformers datasets evaluate accelerate torch scikit-learn rouge_score

# ✅ Enable GPU
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🔍 Using device: {device}")


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.3 MB/s eta 0:00:00


In [4]:
from datasets import load_dataset

# ✅ Load LEDGAR Dataset from Hugging Face
ledgar_dataset = load_dataset("lex_glue", "ledgar")

# ✅ Print Sample Data
print("\n🔍 Sample LEDGAR Entry:")
print(ledgar_dataset["train"][0])

# ✅ Extract Label Mappings
labels_ids2names = {idx: label_name for idx, label_name in enumerate(ledgar_dataset["train"].features["label"].names)}
print("\n📌 LEDGAR Label Mapping (ID → Clause Category):")
print(labels_ids2names)

# Print first few samples from the dataset
print(ledgar_dataset["train"][0])
print("\nDataset Features:")
print(ledgar_dataset["train"].features)

print("Train columns:", ledgar_dataset["train"].column_names)




ModuleNotFoundError: No module named 'datasets'

In [3]:
from transformers import AutoTokenizer
import numpy as np
#old
# ✅ Load Legal-BERT Tokenizer
MODEL_NAME = "nlpaueb/legal-bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def preprocess_data(dataset):
    """Tokenizes LEDGAR dataset and ensures labels are formatted correctly for PyTorch/TensorFlow."""

    def tokenize_function(examples):
        encoding = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

        # ✅ Ensure labels remain as integers
        encoding["label"] = examples["label"]  # No need to convert; it's already an integer

        return encoding

    return dataset.map(tokenize_function, batched=True, remove_columns=["text"], num_proc=1)  # ✅ Avoid parallel processing issues

# ✅ Tokenize LEDGAR dataset
tokenized_ledgar = preprocess_data(ledgar_dataset)

# ✅ Create train-validation split
train_test_split = tokenized_ledgar["train"].train_test_split(test_size=0.1)
tokenized_ledgar["train"], tokenized_ledgar["validation"] = train_test_split["train"], train_test_split["test"]

print("✅ Tokenization complete! Train size:", len(tokenized_ledgar["train"]), "Validation size:", len(tokenized_ledgar["validation"]))


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/222k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/60000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

✅ Tokenization complete! Train size: 54000 Validation size: 6000


In [1]:
MODEL_NAME = "nlpaueb/legal-bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_function(examples):
    # Tokenize the text with fixed max_length, padding, and truncation.
    encoding = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)
    # Create a new "labels" field based on the existing "label" field.
    encoding["labels"] = examples["label"]
    return encoding

# Map the tokenization function (batched=True returns lists for each key).
tokenized_ledgar = ledgar_dataset.map(tokenize_function, batched=True, num_proc=1)
# Now remove the original "text" and "label" columns.
tokenized_ledgar = tokenized_ledgar.remove_columns(["text", "label"])


NameError: name 'AutoTokenizer' is not defined

In [2]:
if "validation" not in tokenized_ledgar:
    split_data = tokenized_ledgar["train"].train_test_split(test_size=0.1)
    tokenized_ledgar["train"] = split_data["train"]
    tokenized_ledgar["validation"] = split_data["test"]

print("Train size:", len(tokenized_ledgar["train"]), "Validation size:", len(tokenized_ledgar["validation"]))


NameError: name 'tokenized_ledgar' is not defined

In [3]:
# Retrieve the number of labels using the original dataset.
num_labels = len(ledgar_dataset["train"].features["label"].names)
print("Number of labels:", num_labels)

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels)


NameError: name 'ledgar_dataset' is not defined

In [24]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir="./legal-bert-ledgar",
    evaluation_strategy="epoch",
    save_strategy="epoch",  # Ensure save strategy matches eval strategy
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_steps=50,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)



In [25]:
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted")
    return {"accuracy": accuracy["accuracy"], "f1": f1["f1"]}


In [26]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ledgar["train"],
    eval_dataset=tokenized_ledgar["validation"],
    compute_metrics=compute_metrics,
)

trainer.train()
model.save_pretrained("./legal-bert-ledgar-final")
tokenizer.save_pretrained("./legal-bert-ledgar-final")


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.660600,0.614421,0.833700,0.819515
2,0.469600,0.553977,0.864600,0.857875
3,0.322200,0.556029,0.875500,0.870367


('./legal-bert-ledgar-final/tokenizer_config.json',
 './legal-bert-ledgar-final/special_tokens_map.json',
 './legal-bert-ledgar-final/vocab.txt',
 './legal-bert-ledgar-final/added_tokens.json',
 './legal-bert-ledgar-final/tokenizer.json')

In [27]:
# Evaluate the model on the validation set using the Trainer.
eval_results = trainer.evaluate()
print("Evaluation Metrics:")
print("Accuracy: {:.4f}".format(eval_results["eval_accuracy"]))
print("F1 Score: {:.4f}".format(eval_results["eval_f1"]))
from evaluate import load
precision_metric = load("precision")
recall_metric = load("recall")

def compute_extended_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted")["f1"]
    precision = precision_metric.compute(predictions=predictions, references=labels, average="weighted")["precision"]
    recall = recall_metric.compute(predictions=predictions, references=labels, average="weighted")["recall"]
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# You can call compute_extended_metrics with your validation data manually:
logits = trainer.predict(tokenized_ledgar["validation"]).predictions
labels = tokenized_ledgar["validation"]["labels"]
ext_metrics = compute_extended_metrics((logits, labels))
print("Extended Evaluation Metrics:")
for metric, value in ext_metrics.items():
    print(f"{metric.capitalize()}: {value:.4f}")

    import pandas as pd
from datasets import Dataset

# Load the real-world test data from the sample file.
# This example assumes each line is "clause_text<TAB>label", e.g.:
# "Clause text here...    97"
real_world_data = []
sample_file_path = "/content/sample_data/master_services_agreement.txt"
with open(sample_file_path, "r", encoding="utf-8") as f:
    for line in f:
        parts = line.strip().split("\t")
        # Ensure that both clause text and label are provided.
        if len(parts) == 2:
            text, label = parts
            try:
                label = int(label)
            except ValueError:
                continue  # Skip lines with invalid label format.
            real_world_data.append({"text": text, "label": label})

# Check that we have data.
if not real_world_data:
    raise ValueError("No valid real-world test data found. Ensure your file has the expected format.")

# Create a Hugging Face Dataset from the data.
real_world_dataset = Dataset.from_list(real_world_data)

# Tokenize the real-world test dataset.
def tokenize_real_world(example):
    encoding = tokenizer(example["text"], padding="max_length", truncation=True, max_length=512)
    encoding["labels"] = example["label"]
    return encoding

tokenized_real_world = real_world_dataset.map(tokenize_real_world, batched=False)
tokenized_real_world = tokenized_real_world.remove_columns(["text"])  # Remove raw text column if desired

# Use the trainer to get predictions.
predictions_output = trainer.predict(tokenized_real_world)
logits = predictions_output.predictions
predicted_labels = np.argmax(logits, axis=-1)
true_labels = np.array(tokenized_real_world["labels"])

# Compute evaluation metrics.
accuracy = accuracy_metric.compute(predictions=predicted_labels, references=true_labels)["accuracy"]
f1 = f1_metric.compute(predictions=predicted_labels, references=true_labels, average="weighted")["f1"]
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
precision = precision_metric.compute(predictions=predicted_labels, references=true_labels, average="weighted")["precision"]
recall = recall_metric.compute(predictions=predicted_labels, references=true_labels, average="weighted")["recall"]

print("Real-World Test Metrics:")
print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1 Score: {:.4f}".format(f1))



Evaluation Metrics:
Accuracy: 0.8755
F1 Score: 0.8704
Extended Evaluation Metrics:
Accuracy: 0.8755
Precision: 0.8774
Recall: 0.8755
F1: 0.8704


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Map:   0%|          | 0/73 [00:00<?, ? examples/s]

Real-World Test Metrics:
Accuracy: 0.4247
Precision: 0.8801
Recall: 0.4247
F1 Score: 0.5415


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [4]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

# ✅ Load Pretrained Legal-BERT Model
num_labels = len(ledgar_dataset["train"].features["label"].names)  # Get total number of legal clause categories
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels).to(device)

# ✅ Define Training Arguments
training_args = TrainingArguments(
    output_dir="./models/legal-bert-ledgar",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,
    num_train_epochs=3,  # Adjust based on performance
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs/legal-bert-ledgar",
    logging_steps=50,
    save_total_limit=2,
    fp16=True,  # ✅ Enable Mixed Precision for Speed
)

# ✅ Trainer Setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ledgar["train"],
    eval_dataset=tokenized_ledgar["validation"]
)

# ✅ Start Training
trainer.train()

# ✅ Save Fine-Tuned Model
model.save_pretrained("./models/legal-bert-ledgar")
tokenizer.save_pretrained("./models/legal-bert-ledgar")

print("✅ Fine-Tuning Complete for Legal-BERT on LEDGAR!")


pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlpaueb/legal-bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kanishk6101 (kanishk6101-purdue-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.636300,0.609041
2,0.301000,0.481940


✅ Fine-Tuning Complete for Legal-BERT on LEDGAR!


In [5]:
import evaluate
import numpy as np
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer

# ✅ Load Fine-Tuned Legal-BERT Model
model = AutoModelForSequenceClassification.from_pretrained("./models/legal-bert-ledgar").to(device)
tokenizer = AutoTokenizer.from_pretrained("./models/legal-bert-ledgar")

# ✅ Load Preprocessed Validation Data
eval_dataset = tokenized_ledgar["validation"]

# ✅ Define Trainer
training_args = TrainingArguments(
    output_dir="./results",
    per_device_eval_batch_size=16,
    logging_dir="./logs"
)

trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=eval_dataset
)

# ✅ Evaluate Model
results = trainer.evaluate()
print("\n📊 Raw Evaluation Results:", results)

# ✅ Load Evaluation Metrics
accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall = evaluate.load("recall")
f1 = evaluate.load("f1")

# ✅ Generate Predictions & References
predictions_output = trainer.predict(eval_dataset)
predictions = np.argmax(predictions_output.predictions, axis=1)  # Get the predicted label
references = np.array(eval_dataset["label"])  # True labels from dataset

# ✅ Compute Classification Metrics
classification_metrics = {
    "accuracy": accuracy.compute(predictions=predictions, references=references)["accuracy"] * 100,
    "precision": precision.compute(predictions=predictions, references=references, average="macro")["precision"] * 100,
    "recall": recall.compute(predictions=predictions, references=references, average="macro")["recall"] * 100,
    "f1": f1.compute(predictions=predictions, references=references, average="macro")["f1"] * 100
}

# ✅ Print Updated Classification Metrics
print("\n📊 Updated Classification Metrics (Percentage):")
for key, value in classification_metrics.items():
    print(f"✅ {key.capitalize()}: {value:.2f}%")



📊 Raw Evaluation Results: {'eval_loss': 0.4819398522377014, 'eval_model_preparation_time': 0.003, 'eval_runtime': 11.6996, 'eval_samples_per_second': 512.84, 'eval_steps_per_second': 32.053}



📊 Updated Classification Metrics (Percentage):
✅ Accuracy: 87.32%
✅ Precision: 80.82%
✅ Recall: 78.39%
✅ F1: 78.87%


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [8]:
# 🚀 Google Colab: Test Legal-BERT on LEDGAR with a Sample Contract

import os
import torch
import evaluate
import numpy as np
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
from nltk.tokenize import sent_tokenize
import nltk

# ✅ Download NLTK Tokenizer
nltk.download('punkt_tab')

# ✅ Enable GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🔍 Using device: {device}")

# ✅ Define LEDGAR Label Mapping
LABELS_ID2NAME = {
    0: "Adjustments", 1: "Agreements", 2: "Amendments", 3: "Anti-Corruption Laws", 4: "Applicable Laws",
    5: "Approvals", 6: "Arbitration", 7: "Assignments", 8: "Assigns", 9: "Authority", 10: "Authorizations",
    11: "Base Salary", 12: "Benefits", 13: "Binding Effects", 14: "Books", 15: "Brokers", 16: "Capitalization",
    17: "Change In Control", 18: "Closings", 19: "Compliance With Laws", 20: "Confidentiality", 21: "Consent To Jurisdiction",
    22: "Consents", 23: "Construction", 24: "Cooperation", 25: "Costs", 26: "Counterparts", 27: "Death", 28: "Defined Terms",
    29: "Definitions", 30: "Disability", 31: "Disclosures", 32: "Duties", 33: "Effective Dates", 34: "Effectiveness",
    35: "Employment", 36: "Enforceability", 37: "Enforcements", 38: "Entire Agreements", 39: "Erisa", 40: "Existence",
    41: "Expenses", 42: "Fees", 43: "Financial Statements", 44: "Forfeitures", 45: "Further Assurances", 46: "General",
    47: "Governing Laws", 48: "Headings", 49: "Indemnifications", 50: "Indemnity", 51: "Insurances", 52: "Integration",
    53: "Intellectual Property", 54: "Interests", 55: "Interpretations", 56: "Jurisdictions", 57: "Liens", 58: "Litigations",
    59: "Miscellaneous", 60: "Modifications", 61: "No Conflicts", 62: "No Defaults", 63: "No Waivers", 64: "Non-Disparagement",
    65: "Notices", 66: "Organizations", 67: "Participations", 68: "Payments", 69: "Positions", 70: "Powers", 71: "Publicity",
    72: "Qualifications", 73: "Records", 74: "Releases", 75: "Remedies", 76: "Representations", 77: "Sales", 78: "Sanctions",
    79: "Severability", 80: "Solvency", 81: "Specific Performance", 82: "Submission To Jurisdiction", 83: "Subsidiaries",
    84: "Successors", 85: "Survival", 86: "Tax Withholdings", 87: "Taxes", 88: "Terminations", 89: "Terms", 90: "Titles",
    91: "Transactions With Affiliates", 92: "Use Of Proceeds", 93: "Vacations", 94: "Venues", 95: "Vesting",
    96: "Waiver Of Jury Trials", 97: "Waivers", 98: "Warranties", 99: "Withholdings"
}

# ✅ Load Fine-Tuned Legal-BERT Model
model_path = "./models/legal-bert-ledgar"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path).to(device)

# ✅ Load Sample Contract File
contract_path = "/content/sample_data/master_services_agreement.txt"

with open(contract_path, "r", encoding="utf-8") as file:
    contract_text = file.read()

# ✅ Tokenize Contract into Clauses (Sentences)
clauses = sent_tokenize(contract_text)
print(f"✅ Extracted {len(clauses)} clauses from contract.")

# ✅ Manually Defined Ground Truth Labels (Modify as Needed)
ground_truth_labels = [
    "Agreements", "Organizations", "Duties", "Payments", "Fees", "Confidentiality",
    "Confidentiality", "Confidentiality", "Intellectual Property", "Intellectual Property",
    "Terminations", "Governing Laws", "Arbitration", "Miscellaneous", "Miscellaneous"
] + ["Miscellaneous"] * (len(clauses) - 15)  # Extend with default labels

# ✅ Classify Each Clause with Legal-BERT
def classify_clause(clause):
    """Classifies a single clause using fine-tuned Legal-BERT."""
    inputs = tokenizer(clause, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        prediction = torch.argmax(outputs.logits, dim=1).item()
    return LABELS_ID2NAME[prediction]  # Convert numeric label to category name

# ✅ Get Predictions for All Clauses
classified_clauses = [(clause, classify_clause(clause)) for clause in clauses]

# ✅ Print Classified Clauses with Expected Labels
print("\n🔍 Classified Clauses:")
for i, (clause, predicted_label) in enumerate(classified_clauses):
    expected_label = ground_truth_labels[i] if i < len(ground_truth_labels) else "Miscellaneous"
    print(f"📌 {clause} → {predicted_label} (Expected: {expected_label})")

# ✅ Compute Evaluation Metrics
accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall = evaluate.load("recall")
f1 = evaluate.load("f1")

# ✅ Convert Labels to Numeric IDs for Metric Calculation
predictions = np.array([list(LABELS_ID2NAME.values()).index(label) for _, label in classified_clauses])
references = np.array([list(LABELS_ID2NAME.values()).index(label) for label in ground_truth_labels])

# ✅ Compute Classification Metrics
classification_metrics = {
    "accuracy": accuracy.compute(predictions=predictions, references=references)["accuracy"] * 100,
    "precision": precision.compute(predictions=predictions, references=references, average="macro")["precision"] * 100,
    "recall": recall.compute(predictions=predictions, references=references, average="macro")["recall"] * 100,
    "f1": f1.compute(predictions=predictions, references=references, average="macro")["f1"] * 100
}

print("\n📊 Updated Classification Metrics (Percentage):")
for key, value in classification_metrics.items():
    print(f"✅ {key.capitalize()}: {value:.2f}%")

# ✅ Summarize Contract Using BART
summary_pipeline = pipeline("summarization", model="facebook/bart-large-cnn", device=0 if torch.cuda.is_available() else -1)
summary = summary_pipeline(contract_text, max_length=200, min_length=50, do_sample=False)[0]['summary_text']

print("\n📝 Contract Summary:")
print(summary)


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


🔍 Using device: cuda
✅ Extracted 36 clauses from contract.

🔍 Classified Clauses:
📌 MASTER SERVICES AGREEMENT
This Master Services Agreement (“Agreement”) is entered into as of March 19, 2025, by and between:

AlphaTech Solutions Inc., a Delaware corporation with its principal place of business at 1000 Enterprise Blvd, San Francisco, CA 94105 (“Company”),
and Innovate Consulting LLC, a limited liability company registered in New York, with its principal office at 500 Strategy Avenue, New York, NY 10001 (“Consultant”). → Agreements (Expected: Agreements)
📌 1. → General (Expected: Organizations)
📌 SCOPE OF SERVICES
1.1 Services Provided: Consultant shall provide technology consulting services, including but not limited to:

IT infrastructure assessment
Software architecture design
Data security compliance
1.2 Deliverables: Consultant agrees to submit all deliverables in electronic format by the deadlines specified in each Statement of Work (SOW). → General (Expected: Duties)
📌 1.3 Modifi

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



📊 Updated Classification Metrics (Percentage):
✅ Accuracy: 13.89%
✅ Precision: 8.70%
✅ Recall: 5.10%
✅ F1: 5.64%


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0



📝 Contract Summary:
This Master Services Agreement (“Agreement”) is entered into as of March 19, 2025, by and between:.AlphaTech Solutions Inc., a Delaware corporation with its. principal place of business at 1000 Enterprise Blvd, San Francisco, CA 94105 (‘Company’),. Innovate Consulting LLC, a limited liability company registered in New York, with. its principal office at 500 Strategy Avenue, New York,. NY 10001 (’Consultant’)
